In [1]:
%pylab inline
import igraph
import networkx
from itertools import combinations
from scipy.sparse import lil_matrix, diags
from scipy.sparse.linalg import eigsh

Populating the interactive namespace from numpy and matplotlib


### Read edge-list from binary file:

In [2]:
path = "expl_metabolic.bin"
EL = fromfile(path, dtype = int32).reshape((-1, 2))

### Make indexing after 0 for igraph:

In [3]:
UEL = unique(EL)
EL = vectorize(dict(zip(UEL, arange(len(UEL)))).__getitem__)(EL)

### Create igraph object:

In [4]:
g = igraph.Graph()
g.add_vertices(unique(EL))
g.add_edges(EL)

### Create NetworkX object:

In [5]:
gnx = networkx.Graph()
gnx.add_nodes_from(unique(EL))
gnx.add_edges_from(EL)

### Calculate the number of $\textit{nodes}$ and $\textit{edges}$:

In [6]:
nodes = g.vcount()
edges = g.ecount()
print("Number of nodes: ", nodes) # number of vertices
print("Number of edges: ", edges) # number of edges

Number of nodes:  1121
Number of edges:  3845


### Calculate the degree distribution ($P_k$):

In [7]:
deg = g.degree()
k, Nk = unique(deg, return_counts=True)
Pk = Nk/sum(Nk)

### Calculate $\textit{Shanon entropy}$ ($-\sum_k P_k \cdot \log{P_k}$) as proposed in [1]:

In [8]:
ShanonEntropy = -sum(Pk*log(Pk))
print("Shannon entropy: ", ShanonEntropy)

Shannon entropy:  2.378932968683868


### Calculate the $\textit{Energy}$ ($\log (\prod_k k!^{N_k})$) as proposed in [2]:

In [9]:
Energy = sum(Nk*array([sum([log(i) for i in range(1, j+1)]) for j in k]))
print("Energy: ", Energy)

Energy:  15905.526641072065


### Calculate the  $\textit{Entropy}$ ($\log(\frac{(2L)!}{\prod_k (k \cdot N_k)!})$) as proposed in [2]:

In [10]:
twoL = g.ecount()
Entropy = sum([log(j) for j in range(1, twoL+1)]) - sum([sum([log(t) for t in range(1, k[r]*Nk[r])]) 
                                                         for r in range(len(k))]) 
print("Entropy: ", Entropy)

Entropy:  -7267.711254442438


### Calculate the  $\textit{Spearman correlation coefficient}$ as proposed in [3]:

In [11]:
def SpearmanCorrelationCoefficient(graph):
    u, c = unique(array(networkx.degree(graph))[:, 1], return_counts=True)
    kmin = min(u)
    kmax = max(u)
    Nk = dict(zip(u, c))
    for i in setdiff1d(arange(kmin, kmax), u):
        Nk[i] = 0
    NNN = sum([i*Nk[i] for i in range(kmin, kmax+1)])
    X1 = []
    X2 = []
    for e in graph.edges():
        k1 = graph.degree(e[0])
        X1.append((sum([i*Nk[i] for i in range(kmin, k1)]) + 0.5*k1*Nk[k1])/NNN)
        k2 = graph.degree(e[1])
        X2.append((sum([i*Nk[i] for i in range(kmin, k2)]) + 0.5*k2*Nk[k2])/NNN)
    X1 = array(X1)
    X2 = array(X2)
    rS = (mean(X1*X2) - mean(X1)*mean(X2))/(std(X1)*std(X2))
    return rS
SpearmanCorr = SpearmanCorrelationCoefficient(gnx)
print("Spearman correlation coefficient: ", SpearmanCorr)

Spearman correlation coefficient:  -0.20311941439819917


### Calculate the  $\textit{eigenvalue distribution of the transition probability matrix (TPM)}$ as proposed in [4]:

In [12]:
def GiveMomAndMinMax(DISTRIBUTION):
    # Function giving the following properties of a distribution:
    # min, min-1, max-1, max, mom1, mom2, mom3, mom4
    MIN_MAX_MOM = []
    DISTRIBUTION = DISTRIBUTION[argsort(DISTRIBUTION)]
    MIN_MAX_MOM.extend(list(DISTRIBUTION[:2]))
    MIN_MAX_MOM.extend(list(DISTRIBUTION[-3:-1]))
    DISTRIBUTION = DISTRIBUTION / DISTRIBUTION.max()
    for i in range(1, 5):
        MIN_MAX_MOM.append((DISTRIBUTION**i).sum() / len(DISTRIBUTION))
    return MIN_MAX_MOM

# Creating the Transition probability matrix in sparse format (to optimize memory usage):
Lun = len(unique(EL)) + 1
row = EL[:, 0]
col = EL[:, 1]
data = ones(len(row)).astype("int32")
m = lil_matrix((Lun, Lun), dtype = "int32") #more efficient
m[row, col] = data
m[col, row] = data
m = m.tocsr()
ondiag = array((m.sum(axis = 1) == 0).astype("int32").flatten())[0]
ondiag = diags(ondiag, format = "csr")
m = m + ondiag
m = diags(1/m.sum(axis = 1).A.ravel()).dot(m)

# Calculating the eigenvalue distribution:
eval_Distr = eigsh(m, k = m.shape[0] - 1, return_eigenvectors = False)

# Calculating the extremal values and moments for global indicators:
eval_Distr = GiveMomAndMinMax(eval_Distr)
print("Eigenvalue distribution of the TPM (min, min-1, max-1, max, mom1, mom2, mom3, mom4): ", eval_Distr)

Eigenvalue distribution of the TPM (min, min-1, max-1, max, mom1, mom2, mom3, mom4):  [-0.8944394013704453, -0.8313144903720628, 0.8912018761157647, 0.9916760056383392, 0.0008913684743778671, 0.14067107757684102, 0.002796156808739711, 0.04182204719921925]


## References
1. Costa, L. da F., Rodrigues, F. A., Travieso, G., & Villas Boas, P. R. (2007). Characterization of complex networks: A survey of measurements. Advances in Physics, 56(1), 167–242.
2. Ginestra Bianconi: “Degree distribution of complex networks from statistical mechanics principles”, 2006; [http://arxiv.org/abs/cond-mat/0606365 arXiv:cond-mat/0606365].
3. Litvak, N., & van der Hofstad, R. (2013). Uncovering disassortativity in large scale-free networks. Physical Review E, 87(2).
4. de Arruda, G. F., Rodrigues, F. A., & Moreno, Y. (2018). Fundamentals of spreading processes in single and multilayer complex networks. Physics Reports, 756, 1–59.